In [2]:
import duckdb
import pandas as pd
import numpy as np
from pathlib import Path

In [10]:
root_path = "../"
dbname = "compe_02"
# root_path = root_path.as_posix() + '/'

# Connect to DuckDB database
con = duckdb.connect(root_path+'duckdb/'+ dbname + ".duckdb")    

meses_train = [202012]
meses_test = [202108]

meses = meses_train + meses_test
meses = ', '.join(map(str, meses))


query = (f"""
        SELECT *
        FROM {dbname}
        WHERE {dbname}.foto_mes IN ({meses});""")

# Execute the join query and fetch the result as a Pandas DataFrame
data = con.execute(query).fetchdf()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
def psi(expected, actual, buckets=255):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [14]:
psi_results = []
for column in data.columns:
  print(column)
  if column not in ['foto_mes', 'clase_ternaria']:
    train_variable = data.loc[data['foto_mes'].isin(meses_train),column]
    print("Train: ", train_variable[:5])
    score_variable =  data.loc[data['foto_mes'].isin(meses_test),column]
    print("Score: ", score_variable[:5])
    psi_value = psi(train_variable, score_variable)
    print("PSI: ", psi_value)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)
psi_df


C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\3242874900.py:4: RuntimeWarning: invalid value encountered in divide
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\3242874900.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\3242874900.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\3242874900.py:4: RuntimeWarning: invalid value encountered in divide
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\3242874900.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppDat

AttributeError: 'float' object has no attribute 'left'

In [ ]:

def preparar_data(dbname, mes_train, mes_test, drop_cols=None, sampling = 1):

    print('Preparando data...')

 

    data = data.drop(['numero_de_cliente_1', 'foto_mes_1', 'clase_ternaria_1'], axis=1)

    # data = pd.read_parquet(dataset_path + dataset_file)

    data['clase_peso'] = 1.0

    data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
    data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

    data['clase_binaria1'] = 0
    data['clase_binaria2'] = 0
    data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
    data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

#   data = data.loc[data['foto_mes'].isin(mes_train + mes_test)]

    data_continua = data[data['clase_ternaria'] == 'CONTINUA']

    data_continua = data_continua.groupby('foto_mes').sample(frac=sampling, random_state=42)

    data = pd.concat([data_continua, data[data['clase_ternaria'] != 'CONTINUA']])

    print(data.shape)

    print(data.head())

    del data_continua

    train_data = data.loc[data['foto_mes'].isin(mes_train)]
    test_data = data[data['foto_mes'].isin(mes_test)]

    X_train = train_data.drop(['numero_de_cliente','clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_train_binaria1 = train_data['clase_binaria1']
    y_train_binaria2 = train_data['clase_binaria2']
    w_train = train_data['clase_peso']

    X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_test_binaria1 = test_data['clase_binaria1']
    y_test_class = test_data['clase_ternaria']
    w_test = test_data['clase_peso']

    print('Data preparada')

    return X_train, y_train_binaria1, y_train_binaria2, w_train, X_test, y_test_class, y_test_binaria1, w_test


